In [1]:
from balm.data import load_dataset, DataCollator
from balm.models import (
    BalmForMaskedLM,
    BalmMoEForMaskedLM,
    BalmExpertChoiceMoEForMaskedLM,
    BalmHybridMoEForMaskedLM,
)
from balm.tokenizer import Tokenizer
from balm.train import Trainer

# from datasets import load_dataset
# from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, EsmTokenizer

import os

import wandb



In [2]:
tokenizer = Tokenizer(vocab="./vocab.json")
# tokenizer = EsmTokenizer.from_pretrained("./vocab.json")



In [3]:
def remove_sep(txt):
    return txt.replace("</s>", "<cls><cls>")


data_files = {
    "train": "./balm/test_data/test.txt",
    "test": "./balm/test_data/test_1k.txt",
    "eval": "./balm/test_data/test_1k.txt",
}

dataset = load_dataset("text", data_files=data_files, preprocess_fn=remove_sep)

In [4]:
tokenized_dataset = dataset.map(
    lambda x: tokenizer(
        x["text"],
        padding=True,
        truncation=True,
        max_length=320,
    ),
    remove_columns="text"
)

Encoding:   0%|          | 0/66792 [00:00<?, ?it/s]

Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

Encoding:   0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
collator = DataCollator(tokenizer=tokenizer)
# collator = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, 
#     mlm=True,
#     mlm_probability=0.15,
# )



In [6]:
# # matched to ESM-2 8M
# model = BalmForMaskedLM(
#     embed_dim=320,
#     ffn_dim=320*4,
#     num_layers=6,
#     num_heads=20,
#     vocab_size=tokenizer.vocab_size,
# )

# # matched to ESM-2 8M
# model = BalmMoEForMaskedLM(
#     embed_dim=320,
#     ffn_dim=320*4,
#     num_experts=8,
#     num_shared_experts=0,
#     num_layers=6,
#     num_heads=20,
#     alternate_sparsity=True,
#     router_top_k=1,
#     expert_capacity=128,
#     router_z_loss_coef=0.01,
#     router_aux_loss_coef=0.01,
#     vocab_size=tokenizer.vocab_size,
# )

# matched to ESM-2 8M
model = BalmExpertChoiceMoEForMaskedLM(
    embed_dim=320,
    ffn_dim=320 * 4,
    num_experts=8,
    num_shared_experts=0,
    num_layers=6,
    num_heads=20,
    alternate_sparsity=False,
    expert_capacity=128,
    router_z_loss_coef=0.01,
    vocab_size=tokenizer.vocab_size,
)

In [7]:
model.num_parameters

41935073

In [8]:
trainer = Trainer(
    model=model,
    data_collator=collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"],
    epochs=1,
    logging_steps=5,
    eval_steps=10,
    warmup_steps=10,
    per_device_train_batch_size=32,
    # use_cpu=True,
    use_wandb=True,
    wandb_project="test_wandb_logging",
    # wandb_entity="bryanbriney",
    run_name="test_wandb_logging_003",
)

# training_args = TrainingArguments(
#     output_dir="~/Desktop/training",
#     logging_dir="~/Desktop/training/log",
#     per_device_train_batch_size=32,
#     learning_rate=1e-4,
#     max_steps=2000,
#     gradient_accumulation_steps=1,
#     logging_steps=10,
#     eval_steps=50,
#     warmup_steps=100,
#     use_cpu=True,
#     report_to="none"
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     data_collator=collator,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["eval"],
# )




In [9]:
os.environ["WANDB_PROJECT"] = trainer.run_name
wandb.login()

wandb: Currently logged in as: bryanbriney (thebrineylab). Use `wandb login --relogin` to force relogin


True

In [10]:
trainer.train()

wandb: Currently logged in as: bryanbriney. Use `wandb login --relogin` to force relogin


Training:   0%|          | 0/2087 [00:00<?, ?step/s]

step 5     | loss: 3.0656 | MLM loss: 3.0134 | router z-loss: 0.0522 | lr: 0.000200
step 10    | loss: 2.8217 | MLM loss: 2.7818 | router z-loss: 0.0399 | lr: 0.000400


Evaluating:   0%|          | 0/31 [00:00<?, ?step/s]

<< EVAL >> | loss: 2.9026 | accuracy: 0.1677 | perplexity: 16.0625
step 15    | loss: 2.6807 | MLM loss: 2.6556 | router z-loss: 0.0251 | lr: 0.000399
step 20    | loss: 2.5866 | MLM loss: 2.5709 | router z-loss: 0.0157 | lr: 0.000398


...

Thread SenderThread:
Traceback (most recent call last):
  File "/Users/bryanbriney/conda/lib/python3.11/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/Users/bryanbriney/conda/lib/python3.11/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/Users/bryanbriney/conda/lib/python3.11/site-packages/wandb/sdk/internal/internal.py", line 328, in _process
    self._sm.send(record)
  File "/Users/bryanbriney/conda/lib/python3.11/site-packages/wandb/sdk/internal/sender.py", line 389, in send
    send_handler(record)
  File "/Users/bryanbriney/conda/lib/python3.11/site-packages/wandb/sdk/internal/sender.py", line 411, in send_request
    send_handler(record)
  File "/Users/bryanbriney/conda/lib/python3.11/site-packages/wandb/sdk/internal/sender.py", line 1160, in send_request_summary_record
    self._update_summary_record(record.request.summary_record.summary)
  File "/Users/bryanbriney/conda/lib/py

step 1295  | loss: 0.5281 | MLM loss: 0.5278 | router z-loss: 0.0003 | lr: 0.000153


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x2de2df490>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe